In [ ]:
# 코랩 프로에서만 가능. 일반에서 하면 허깅페이스 올리다가 메모리가 터짐

In [ ]:
# savePath = "/content/gdrive/MyDrive/Colab Notebooks/data_edu/16. LLM 파인튜닝/models/llama_tune_{now_str}"

In [ ]:
try:
    import google.colab
    inColab = True
except ImportError:
    inColab = False

In [ ]:
if inColab == True:
    !pip install -U pandas==2.2.2 numpy==2.0.2 scipy==1.14.1 accelerate==1.6.0 peft==0.15.2 bitsandbytes==0.45.5 transformers==4.51.3 trl==0.16.1 datasets==3.5.0 tensorboard==2.19.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from transformers import AutoConfig,AutoModel
import torch
from peft import PeftModel, PeftConfig

In [ ]:
if inColab == True:
    from google.colab import drive
    drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Colab Notebooks/fintech_edu_2025/중간고사/LLM/datasets

In [ ]:
import huggingface_hub
huggingface_hub.login()

# 1. Load base and new model

In [ ]:
# from transformers import AutoConfig, AutoModel, AutoTokenizer
# config = AutoConfig.from_pretrained("your model name", revision=revision)
# model = AutoModel.from_pretrained("your model name", revision=revision)
# tokenizer = AutoTokenizer.from_pretrained("your model name", revision=revision)

### ★★★ 수정 포인트 ★★★

In [ ]:
## base 모델
base_model = "beomi/Llama-3-Open-Ko-8B"
## 파인튜닝한 모델 저장 위치 설정
new_model = "./models/llama_tune_common"

In [ ]:
# # 오프로드 디렉토리 지정
# offload_dir = './offload'

# # 오프로드 디렉토리가 없다면 생성
# if not os.path.exists(offload_dir):
#     os.makedirs(offload_dir)

In [ ]:
### 베이스모델 불러오기
baseModel = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map= "auto"
    # device_map= {"": 0}
)

### 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# PEFT 설정 로드
peft_config = PeftConfig.from_pretrained(new_model)
print(peft_config)

# 베이스모델에 어댑터 적용
model = PeftModel.from_pretrained(baseModel, new_model)

LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='beomi/Llama-3-Open-Ko-8B', revision=None, inference_mode=True, r=64, target_modules={'v_proj', 'q_proj'}, exclude_modules=None, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)


### 어뎁터 연결 모델 정상작동 확인

In [ ]:
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    model_kwargs={"torch_dtype": torch.float16},
    truncation=True
)

def extract_response_beomi(question):
    messages = [
        {"role": "system", "content": ""},
        {"role": "user", "content": question},
    ]

    prompt = pipe.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    terminators = [
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipe(
        prompt,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        num_return_sequences=1
    )

    generated_text = outputs[0]['generated_text']
    response_lines = generated_text.strip().split('\n')
    meaningful_response = response_lines[-1]

    return meaningful_response
question = "임대차계약 종료 후의 인도 의무는 어떻게 발생하나요?"
response = extract_response_beomi(question)
print(response)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

민법 제1019조에 따르면, 임대차계약이 종료되면 임차인은 목적물을 원상으로 복구하여 임대인에게 반환해야 할 의무가 있습니다. 구체적으로, 임대차계약 종료 시 임차인은 목적물을 정상적인 사용 상태로 복구하여 임대인에게 반환해야 하며, 이와 관련한 비용은 임대차계약서에 명시된 바에 따라 임차인이 부담해야 합니다.


In [ ]:
# 어댑터 병합
mergedModel = model.merge_and_unload()

In [ ]:
# set your HF repository
hfAddr = "limjh12/beomi_law"

# save model and tokenizer to HF hub
# 꼭 모델과 해석기를 같이 넣어야 함
mergedModel.push_to_hub(hfAddr)   # 모델
tokenizer.push_to_hub(hfAddr)   # 해석기

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

C:\Users\SMT33\.limjh\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SMT33\.cache\huggingface\hub\models--limjh12--beomi_law. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/limjh12/beomi_law/commit/ec340b41aee9ff7741f64aae66bec7910a65adc8', commit_message='Upload tokenizer', commit_description='', oid='ec340b41aee9ff7741f64aae66bec7910a65adc8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/limjh12/beomi_law', endpoint='https://huggingface.co', repo_type='model', repo_id='limjh12/beomi_law'), pr_revision=None, pr_num=None)

★ 참고 코드

In [ ]:
### 리더보드 올리기전 테스트 코드 (모델 이상여부 확인, 에러 시 등록 X)

In [ ]:
from transformers import AutoConfig, AutoModel, AutoTokenizer
hfAddr = "limjh12/beomi_law"
config = AutoConfig.from_pretrained(hfAddr, revision="main")
model = AutoModel.from_pretrained(hfAddr, revision="main")
tokenizer = AutoTokenizer.from_pretrained(hfAddr, revision="main")

config.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:  71%|#######   | 3.54G/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/53.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/352 [00:00<?, ?B/s]